In [1]:
import pandas as pd
import numpy as np

In [2]:
d0 = pd.read_excel('/home/eduardo/repositorios/IS_Agro/databases/bronze_mapbiomas_brasil_col9_state_municipality.xlsx', sheet_name=1)
d0['geocode'] = d0['geocode'].astype(str)

## ChatGPT

In [3]:
# Anos
seqano = [year for year in range(1985, 2023+1)]

# Sigla
d = d0.copy()
d['state_acronym'] = d0['municipality - state'].str[-2:]

# Seleção de colunas
d = d[['geocode', 'municipality', 'municipality - state', 'state', 'class', 'state_acronym'] + seqano]

In [4]:
# Subconjunto para classe mosaico
dm1 = d[d['class'] == 21].copy()
dm1.set_index('geocode', inplace=True)

# Subconjunto para pastagem, temporárias e perenes
dm2 = d[d['class'].isin([15, 41, 48]) & d['geocode'].isin(dm1.index)].copy()

In [5]:
# Consolidando área total por município
dm2s = dm2.groupby('geocode')[seqano].sum()

frac1 = dm2[seqano].div(dm2s.loc[dm2['geocode']].values)

In [6]:
# Consolidando por estado
dm3 = d.groupby(['state', 'state_acronym', 'class'])[seqano].sum().reset_index()
dm3.set_index(dm3['state_acronym'] + '-' + dm3['class'].astype(str), inplace=True)

dm4 = dm3.loc[dm2['state_acronym'] + '-' + dm2['class'].astype(str)].copy()
dm4['geocode'] = dm2['geocode'].values

dm4s = dm4.groupby('geocode')[seqano].sum()

frac2 = dm4[seqano].div(dm4s.loc[dm4['geocode']].values)

In [7]:
# Reclassificação para mosaico
frac = pd.DataFrame(np.where(dm2s.loc[dm2['geocode']].values != 0, frac1, frac2))

dmtpp = dm2.copy()
dmtpp[seqano] = dm1.loc[dm2['geocode'], seqano].values * frac.values
dmtpp['class_level_2'] = 'Land Use Mosaic'

In [ ]:
# IDs de municípios faltantes
idsgeo = set(dm1.index) - set(dmtpp['geocode'])
idsuf = dm1.loc[dm1.index.isin(idsgeo), 'state_acronym'].unique()
idsmun = dm1.loc[dm1.index.isin(idsgeo), 'municipality'].unique()
idsmuf = dm1.loc[dm1.index.isin(idsgeo), 'municipality - state'].unique()

# Consolidação por estados faltantes
dm5 = dm3[(dm3['state_acronym'].isin(idsuf)) & (dm3['class'].isin([15, 41, 48]))].convert_dtypes()
dm5s = dm5.groupby(['state', 'state_acronym'])[seqano].sum()
dm5s = dm5s.reset_index().set_index('state_acronym')

dm5[seqano] = dm5[seqano].div(dm5s.loc[dm5['state_acronym']][seqano].values)
idrs = [dm5.index.get_loc(x) for x in dm5.index if any(uf in x for uf in idsuf)]

# Removendo ou tratando valores nulos antes de continuar
idsgeo = [x for x in idsgeo if pd.notna(x)]
idsmuf = [x for x in idsmuf if pd.notna(x)]
idsmun = [x for x in idsmun if pd.notna(x)]

dm6 = pd.concat([pd.DataFrame({
    'geocode': list(idsgeo),  # Garantindo que é uma lista sem NaNs
    'municipality - state': list(idsmuf),  # Garantindo que é uma lista sem NaNs
    'municipality': list(idsmun)  # Garantindo que é uma lista sem NaNs
}),
dm5.iloc[idrs]], axis=1)

# Remova as linhas onde 'geocode' é NaN
print(dm6['geocode'].isna().sum())  # Conta a quantidade de NaNs na coluna geocode
dm6_clean = dm6.dropna(subset=['geocode'])
dm6_clean[seqano] = dm6_clean[seqano] * dm1.loc[dm6_clean['geocode'], seqano].values

# Definir 'class_level_2'
dm6_clean['class_level_2'] = 'Land Use Mosaic'

In [ ]:
dm3[(dm3['state_acronym'].isin(idsuf)) & (dm3['class'].isin([15, 41, 48]))]

## Claude

In [3]:
# Anos
seqano = [year for year in range(1985, 2024)]

# Sigla
d = d0.copy()
d['state_acronym'] = d0['municipality - state'].str[-2:]

d = d[['geocode', 'municipality - state', 'municipality', 'state', 'state_acronym'] + list(d.columns[6:51])]

In [4]:
# Subconjunto para classe mosaico
dm1 = d[d['class'] == 21]  # id 3.4. Mosaic of Uses
dm1 = dm1.set_index('geocode')

# Subconjunto para pastagem, temporárias e perenes
dm2 = d[(d['class'].isin([15, 41, 48])) & (d['geocode'].isin(dm1.index))]
dm2s = dm2.groupby('geocode')[seqano].sum().reset_index()
dm2s = dm2s.set_index('geocode')

frac1 = dm2[seqano].values / dm2s.loc[dm2['geocode'], seqano].values
print(f"NaN values in frac1: {np.isnan(frac1).sum()}")

NaN values in frac1: 658


/tmp/ipykernel_72968/2876347537.py:10: RuntimeWarning: invalid value encountered in divide
  frac1 = dm2[seqano].values / dm2s.loc[dm2['geocode'], seqano].values


In [5]:
 # Consolidando por estado
dm3 = d.groupby(['state', 'state_acronym', 'class', 'class_level_0', 'class_level_1', 'class_level_2', 'class_level_3', 'class_level_4'])[seqano].sum().reset_index()
dm3.index = dm3['state_acronym'] + '-' + dm3['class'].astype(str)

dm4 = dm3.loc[dm2['state_acronym'] + '-' + dm2['class'].astype(str)]
dm4['geocode'] = dm2['geocode'].values
dm4s = dm4.groupby('geocode')[seqano].sum().reset_index()
dm4s = dm4s.set_index('geocode')

frac2 = dm4[seqano].values / dm4s.loc[dm4['geocode'], seqano].values
print(f"NaN values in frac2: {np.isnan(frac2).sum()}")
print(f"Zero values in dm4s: {(dm4s == 0).sum().sum()}")

NaN values in frac2: 0
Zero values in dm4s: 0


In [11]:
dm4

,state,state_acronym,class,class_level_0,class_level_1,class_level_2,class_level_3,class_level_4,1985,1986,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,geocode
RO-15,Rondônia,RO,15,Antropic,3. Farming,3.1. Pasture,3.1. Pasture,3.1. Pasture,1.436518e+06,1.785557e+06,...,7.657179e+06,7.811839e+06,7.936922e+06,8.036507e+06,8.207099e+06,8.348657e+06,8.560963e+06,8.809953e+06,9.065506e+06,1100304
RO-41,Rondônia,RO,41,Antropic,3. Farming,3.2. Agriculture,3.2.1. Temporary Crop,3.2.1.5. Other Temporary Crops,6.644980e+03,5.427481e+03,...,8.603885e+04,8.895386e+04,1.030408e+05,1.160173e+05,1.077625e+05,9.336145e+04,4.720241e+04,4.426005e+04,4.151860e+04,1100304
PA-15,Pará,PA,15,Antropic,3. Farming,3.1. Pasture,3.1. Pasture,3.1. Pasture,4.225516e+06,4.544903e+06,...,1.877719e+07,1.909112e+07,1.930993e+07,1.954667e+07,1.993728e+07,2.035945e+07,2.110238e+07,2.171949e+07,2.252251e+07,1502707
PA-41,Pará,PA,41,Antropic,3. Farming,3.2. Agriculture,3.2.1. Temporary Crop,3.2.1.5. Other Temporary Crops,1.350265e+03,2.158133e+02,...,2.982504e+05,3.074526e+05,3.498528e+05,3.697837e+05,3.819599e+05,3.720052e+05,2.185962e+05,1.835019e+05,1.740104e+05,1502707
PA-15,Pará,PA,15,Antropic,3. Farming,3.1. Pasture,3.1. Pasture,3.1. Pasture,4.225516e+06,4.544903e+06,...,1.877719e+07,1.909112e+07,1.930993e+07,1.954667e+07,1.993728e+07,2.035945e+07,2.110238e+07,2.171949e+07,2.252251e+07,1503044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GO-41,Goiás,GO,41,Antropic,3. Farming,3.2. Agriculture,3.2.1. Temporary Crop,3.2.1.5. Other Temporary Crops,7.923293e+05,3.268939e+05,...,9.812363e+05,1.240841e+06,6.195649e+05,4.946186e+05,4.877467e+05,7.396466e+05,5.687255e+05,4.032519e+05,3.856193e+05,5222302
GO-48,Goiás,GO,48,Antropic,3. Farming,3.2. Agriculture,3.2.2. Perennial Crop,3.2.2.4. Other Perennial Crops,2.828506e+03,3.286741e+03,...,1.315394e+02,2.179404e+02,2.890855e+02,3.088144e+02,3.175389e+02,5.821043e+02,5.822748e+02,5.838185e+02,5.836529e+02,5222302
DF-15,Distrito Federal\n,DF,15,Antropic,3. Farming,3.1. Pasture,3.1. Pasture,3.1. Pasture,6.673591e+04,7.327663e+04,...,8.912919e+04,9.090794e+04,9.004152e+04,8.904581e+04,8.896261e+04,8.780424e+04,8.531888e+04,7.975002e+04,7.067023e+04,5300108
DF-41,Distrito Federal\n,DF,41,Antropic,3. Farming,3.2. Agriculture,3.2.1. Temporary Crop,3.2.1.5. Other Temporary Crops,1.013426e+04,6.503165e+02,...,1.903104e+04,7.626423e+03,9.007189e+03,8.000633e+03,5.526149e+03,3.790730e+03,3.615228e+03,3.014473e+03,2.948455e+03,5300108


In [10]:
frac = np.where(dm2s.loc[dm2['geocode'], seqano].values != 0, frac1, frac2)
print(f"NaN values in frac: {np.isnan(frac).sum()}")

dmtpp = dm2.copy()
dmtpp[seqano] = dm1.loc[dm2['geocode'], seqano].values * frac
dmtpp['class_level_2'] = 'Land Use Mosaic'

NaN values in frac: 0


In [ ]:
idsgeo = list(set(dm1.index) - set(dmtpp['geocode']))
idsuf = dm1.loc[dm1.index.isin(idsgeo), 'state_acronym'].unique()
idsmun = dm1.loc[dm1.index.isin(idsgeo), 'municipality'].unique()
idsmuf = dm1.loc[dm1.index.isin(idsgeo), 'municipality - state'].unique()

print(f"Sum equality check: {np.allclose(dm1.loc[~dm1.index.isin(idsgeo), seqano].sum(), dmtpp[seqano].sum())}")

In [ ]:
dm5 = dm3[(dm3['state_acronym'].isin(idsuf)) & (dm3['class'].isin([15, 41, 48]))]
dm5s = dm5.groupby(['state', 'state_acronym'])[seqano].sum().reset_index()
dm5s = dm5s.set_index('state_acronym')

dm5[seqano] = dm5[seqano].values / dm5s.loc[dm5['state_acronym'], seqano].values

idrs = [i for uf in idsuf for i in dm5.index[dm5['state_acronym'] == uf]]

In [ ]:
dm6 = pd.DataFrame({
    'geocode': np.repeat(idsgeo, 3),
    'municipality - state': np.repeat(idsmuf, 3),
    'municipality': np.repeat(idsmun, 3)
})
dm6 = pd.concat([dm6, dm5.loc[idrs].reset_index(drop=True)], axis=1)
dm6[seqano] = dm6[seqano].values * dm1.loc[dm6['geocode'], seqano].values
dm6['class_level_2'] = 'Land Use Mosaic'

print(f"Sum difference check: {np.allclose(dm6[seqano].sum().round(6), dm1.loc[dm1['municipality'] == 'São Caetano do Sul', seqano].sum().round(6))}")

In [ ]:
dmtpp = pd.concat([dmtpp, dm6])

print(f"Final sum equality check: {np.allclose(dm1[seqano].sum(), dmtpp[seqano].sum())}")

In [30]:
ds = pd.concat([d[d['class'] != 21], dmtpp])

In [ ]:
# VERIFICAÇÕES
print(f"Sum equality checks:")
print(f"d vs ds: {np.allclose(d[seqano].sum(), ds[seqano].sum())}")
print(f"d vs d0: {np.allclose(d[seqano].sum().round(6), d0[seqano].sum().round(6))}")
print(f"d0 vs ds: {np.allclose(d0[seqano].sum(), ds[seqano].sum())}")

print(f"Subset sum equality checks:")
print(f"d vs ds (excluding idsgeo): {np.allclose(d[~d['geocode'].isin(idsgeo)][seqano].sum(), ds[~ds['geocode'].isin(idsgeo)][seqano].sum())}")
print(f"d0 vs ds (excluding idsgeo): {np.allclose(d0[~d0['geocode'].isin(idsgeo)][seqano].sum(), ds[~ds['geocode'].isin(idsgeo)][seqano].sum())}")
print(f"d vs ds (all): {np.allclose(d[seqano].sum(), ds[seqano].sum())}")
print(f"d0 vs d (all): {np.allclose(d0[seqano].sum(), d[seqano].sum())}")
print(f"d0 vs ds (all, rounded): {np.allclose(d0[seqano].sum().round(6), ds[seqano].sum().round(6))}")

In [32]:
tbr1 = ds.groupby(['class', 'class_level_0', 'class_level_1', 'class_level_2', 'class_level_3', 'class_level_4'])[seqano].sum().reset_index()
tbr2 = ds.groupby(['class_level_0', 'class_level_1', 'class_level_3'])[seqano].sum().reset_index()

ds2 = ds.groupby(['geocode', 'municipality - state', 'municipality', 'state', 'state_acronym', 'class_level_0', 'class_level_1', 'class_level_3'])[seqano].sum().reset_index()

In [ ]:
ds.reset_index(drop=True)

In [ ]:
ds2